In [1]:
from matplotlib import pyplot as plt
from pyspark.conf import SparkConf
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col
from pyspark.sql.types import IntegerType


In [2]:
# Create PySpark SparkSession
conf: SparkConf = SparkConf().setAppName("ECD_TCC").setMaster("local[*]")
spark: SparkSession = SparkSession.builder.config(conf=conf).getOrCreate()


In [3]:
# Filtrando apenas casos de covid
df_raw = spark.read.option("delimiter", ";").option("header", True).csv("../datasets/raw").where("CLASSI_FIN == 5")


In [4]:
# Selecionando apenas as colunas com fatores de risco
select_cols = ["PUERPERA", "CARDIOPATI", "HEMATOLOGI", "SIND_DOWN", "HEPATICA", "ASMA", "DIABETES", "NEUROLOGIC",
               "PNEUMOPATI", "IMUNODEPRE", "RENAL", "OBESIDADE", "VACINA_COV", "VACINA", "EVOLUCAO"]


In [6]:
# Preenchendo valores nulos como ignorados e normalizando os preenchimentos de fatores de risco
df = df_raw.select(select_cols).where("VACINA_COV <> '12/02/2021'").cache()
for column in select_cols:
    df = df.withColumn(
        column,
        when(col(column) == "1", "Y")
        .when(col(column) == "2", "N")
    )
df = df.withColumnRenamed("EVOLUCAO", "label").where(col("label").isNotNull()).cache()


In [7]:
print("Dataset com os dados relevantes")
pandas_df = df.toPandas()
pandas_df.head()


Dataset com os dados relevantes


,PUERPERA,CARDIOPATI,HEMATOLOGI,SIND_DOWN,HEPATICA,ASMA,DIABETES,NEUROLOGIC,PNEUMOPATI,IMUNODEPRE,RENAL,OBESIDADE,VACINA_COV,VACINA,label
0,None,None,None,None,None,None,Y,None,None,None,None,None,Y,Y,Y
1,None,Y,None,None,None,Y,None,None,Y,None,None,None,None,None,Y
2,N,N,N,N,N,N,Y,N,Y,N,N,Y,N,N,Y
3,N,N,N,N,N,N,Y,N,N,N,N,N,N,None,N
4,None,None,None,None,None,None,None,None,None,None,None,None,Y,None,Y


In [8]:
pandas_df.to_csv("../datasets/processed/df.csv", index=False, header=True)

In [ ]:
# Valores de camadas inciais e finais para o MLP
features_n = len(select_cols) - 1
classes_n = df.select("EVOLUCAO").distinct().count() + 1


AnalysisException: Column 'EVOLUCAO' does not exist. Did you mean one of the following? [HEPATICA, VACINA, ASMA, NEUROLOGIC, RENAL, DIABETES, HEMATOLOGI, OBESIDADE, PUERPERA, VACINA_COV, label, CARDIOPATI, IMUNODEPRE, PNEUMOPATI, SIND_DOWN];
'Project ['EVOLUCAO]
+- Project [PUERPERA#440, CARDIOPATI#456, HEMATOLOGI#472, SIND_DOWN#488, HEPATICA#504, ASMA#520, DIABETES#536, NEUROLOGIC#552, PNEUMOPATI#568, IMUNODEPRE#584, RENAL#600, OBESIDADE#616, VACINA_COV#632, VACINA#648, EVOLUCAO#664 AS label#680]
   +- Project [PUERPERA#440, CARDIOPATI#456, HEMATOLOGI#472, SIND_DOWN#488, HEPATICA#504, ASMA#520, DIABETES#536, NEUROLOGIC#552, PNEUMOPATI#568, IMUNODEPRE#584, RENAL#600, OBESIDADE#616, VACINA_COV#632, VACINA#648, CASE WHEN (EVOLUCAO#126 = 1) THEN Y WHEN (EVOLUCAO#126 = 2) THEN N END AS EVOLUCAO#664]
      +- Project [PUERPERA#440, CARDIOPATI#456, HEMATOLOGI#472, SIND_DOWN#488, HEPATICA#504, ASMA#520, DIABETES#536, NEUROLOGIC#552, PNEUMOPATI#568, IMUNODEPRE#584, RENAL#600, OBESIDADE#616, VA

In [ ]:
# Juntar as variáveis em um vetor de features
assembler = VectorAssembler(inputCols=feature_list, outputCol="features")
assembledDF = assembler.transform(df)


IllegalArgumentException: Data type string of column PUERPERA is not supported.
Data type string of column CARDIOPATI is not supported.
Data type string of column HEMATOLOGI is not supported.
Data type string of column SIND_DOWN is not supported.
Data type string of column HEPATICA is not supported.
Data type string of column ASMA is not supported.
Data type string of column DIABETES is not supported.
Data type string of column NEUROLOGIC is not supported.
Data type string of column PNEUMOPATI is not supported.
Data type string of column IMUNODEPRE is not supported.
Data type string of column RENAL is not supported.
Data type string of column OBESIDADE is not supported.
Data type string of column VACINA_COV is not supported.
Data type string of column VACINA is not supported.

In [ ]:
# Modelos a testar: regressão logística, Floresta aleatória e multilayerperceptron
lr = LogisticRegression().setFamily("binomial")
rf = RandomForestClassifier()
mlp = MultilayerPerceptronClassifier()


In [ ]:
# Pipelines de transformações para a validação cruzada
pipeline_lr = Pipeline(stages=[rf])
pipeline_rf = Pipeline(stages=[rf])
pipeline_mlp = Pipeline(stages=[mlp])


In [ ]:
# Parâmetros para a validação cruzada
paramGrid_lr = (
    ParamGridBuilder()
    .addGrid(lr.regParam, [0.01, 0.1, 0.5, 1.0, 2.0])
    .addGrid(lr.elasticNetParam, [0.0, 0.25, 0.5, 0.75, 1.0])
    .addGrid(lr.maxIter, [1, 5, 10, 20, 50])
    .build()
)
paramGrid_rf = (
    ParamGridBuilder()
    .addGrid(rf.numTrees, [10, 30, 50])
    .addGrid(rf.maxDepth, [5, 10, 15])
    .build()
)
paramGrid_mlp = (
    ParamGridBuilder()
    .addGrid(mlp.layers, [
        [features_n, 3, 3, classes_n],
        [features_n, 4, 4, classes_n],
        [features_n, 3, 3, 3, classes_n],
        [features_n, 4, 4, 4, classes_n],
    ]).build()
)

In [ ]:
# Validador cruzado
crossval_lr = CrossValidator(estimator=pipeline_lr, estimatorParamMaps=paramGrid_lr,
                             evaluator=MulticlassClassificationEvaluator())
crossval_rf = CrossValidator(estimator=pipeline_rf, estimatorParamMaps=paramGrid_rf,
                             evaluator=MulticlassClassificationEvaluator())
crossval_mlp = CrossValidator(estimator=pipeline_mlp, estimatorParamMaps=paramGrid_mlp,
                              evaluator=MulticlassClassificationEvaluator())


In [ ]:
# Separar os dados em treino e teste
taxa_de_treino = 0.00001
(trainingData, testData) = df.randomSplit([taxa_de_treino, 1 - taxa_de_treino])


In [ ]:
# Salvando as massa utilizadas em treino e teste para fazer análises depois de gerar os modelos
trainingData.write.mode("overwrite").parquet("../datasets/training")
testData.write.mode("overwrite").parquet("../datasets/test")

In [ ]:
crossval_lr_model = crossval_lr.fit(assembledDF)

Py4JJavaError: An error occurred while calling o11341.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 477.0 failed 1 times, most recent failure: Lost task 0.0 in stage 477.0 (TID 2762) (192.168.1.113 executor driver): scala.MatchError: [null,1.0,[-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0]] (of class org.apache.spark.sql.catalyst.expressions.GenericRowWithSchema)
	at org.apache.spark.ml.PredictorParams.$anonfun$extractInstances$1(Predictor.scala:81)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:461)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:461)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:461)
	at scala.collection.Iterator$SliceIterator.next(Iterator.scala:273)
	at scala.collection.Iterator.foreach(Iterator.scala:943)
	at scala.collection.Iterator.foreach$(Iterator.scala:943)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1431)
	at scala.collection.gener

In [ ]:
crossval_lr_model.write().save("model/lr")

In [ ]:
# Avaliando o modelo
predictions = cvModel.transform(testData)
evaluator = MulticlassClassificationEvaluator()
rmse = evaluator.evaluate(predictions)

predictions.select("label", "features", "rawPrediction",
                   "probability", "prediction").show()
predictions.select("prediction").distinct().show()

result = predictions.toPandas()

plt.plot(result.label, result.prediction, 'bo')
plt.xlabel('Sobrevivencia')
plt.ylabel('Prediction')
plt.suptitle("Model Performance RMSE: %f" % rmse)
plt.show()


In [ ]:
# Selecionando o melhor modelo
bestPipeline = cvModel.bestModel
bestModel = bestPipeline.stages[2]

importances = bestModel.featureImportances

x_values = list(range(len(importances)))

plt.bar(x_values, importances, orientation='vertical')
plt.xticks(x_values, feature_list, rotation=40)
plt.ylabel('Importance')
plt.xlabel('Feature')
plt.title('Feature Importances')


In [ ]:
print('numTrees - ', bestModel.getNumTrees)
print('maxDepth - ', bestModel.getOrDefault('maxDepth'))
